<a href="https://colab.research.google.com/github/jjcremer/NLP_Masters/blob/main/NLP202/nlp202_assignment4_1542248.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)

In [ ]:
def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

In [ ]:
class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2),
                torch.randn(2, 1, self.hidden_dim // 2))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence, lengths):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        embeds_pack = nn.utils.rnn.pack_padded_sequence(sentence, lengths, batch_first=True, enforce_sorted=False)
        #print()
        #print(embeds_pack.size())
        lstm_out, self.hidden = self.lstm(embeds_pack, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags, lengths):
        feats = self._get_lstm_features(sentence, lengths)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence, lengths):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence, lengths)
        #print(lstm_feats.size)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

In [ ]:
START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 5
HIDDEN_DIM = 4

# Make up some training data
training_data = [(
    "the wall street journal reported today that apple corporation made money".split(),
    "B I I I O O O B I O O".split()
), (
    "georgia tech is a university in georgia".split(),
    "B I O O O O B".split()
)]

training_data = read_training_data()

word_to_ix = {}
for sentence, tags in training_data:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

#tag_to_ix = {"B": 0, "I": 1, "O": 2, START_TAG: 3, STOP_TAG: 4}
tag_to_ix = {'O': 0, 'B-DNA': 1, 'I-DNA': 2, 'B-RNA': 3, 'I-RNA': 4, 'B-protein': 5, 'I-protein': 6,
             'B-cell_line': 7, 'I-cell_line': 8, 'B-cell_type': 9, 'I-cell_type': 10, START_TAG: 11, STOP_TAG: 12}

model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

# Check predictions before training
"""
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
    precheck_tags = torch.tensor([tag_to_ix[t] for t in training_data[0][1]], dtype=torch.long)
    print(model(precheck_sent))
    
# Make sure prepare_sequence from earlier in the LSTM section is loaded
for epoch in range(
        5):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

# Check predictions after training
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
    print(model(precheck_sent))
# We got it!
"""

'\nwith torch.no_grad():\n    precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)\n    precheck_tags = torch.tensor([tag_to_ix[t] for t in training_data[0][1]], dtype=torch.long)\n    print(model(precheck_sent))\n    \n# Make sure prepare_sequence from earlier in the LSTM section is loaded\nfor epoch in range(\n        5):  # again, normally you would NOT do 300 epochs, it is toy data\n    for sentence, tags in training_data:\n        # Step 1. Remember that Pytorch accumulates gradients.\n        # We need to clear them out before each instance\n        model.zero_grad()\n\n        # Step 2. Get our inputs ready for the network, that is,\n        # turn them into Tensors of word indices.\n        sentence_in = prepare_sequence(sentence, word_to_ix)\n        targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long)\n\n        # Step 3. Run our forward pass.\n        loss = model.neg_log_likelihood(sentence_in, targets)\n\n        # Step 4. Compute the loss, g

In [ ]:
def read_training_data(path='train'):
  training_data = []
  tokens = []
  tags = []
  with open(path, 'r') as f:
    row = f.readline()
    while(row):
      row = row.split()
      if row == []:
        training_data.append((tokens, tags))
        tokens = []
        tags = []
      else:
        tokens.append(row[0])
        tags.append(row[1])
      row = f.readline()
  return training_data
#training_data = read_training_data()
#print(training_data[0])

In [ ]:
import torch
from torchtext.legacy import data

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy',
                  tokenizer_language = 'en_core_web_sm')
LABEL = data.LabelField(dtype = torch.float)

In [ ]:
import random
import numpy as np
device = torch.device('cuda')

#train_data, valid_data = train_data.split(random_state = random.seed(SEED))
zipped_train_data = read_training_data()
print(f'Number of training examples: {len(zipped_train_data)}')
#print(f'Number of validation examples: {len(valid_data)}')
#print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 18546


In [ ]:
from torch.utils.data import DataLoader
from functools import partial

def custom_collate_func(batch, window_size, word_to_ix, label_to_ix):
  #print(batch)
  x, y = zip(*batch)

  def pad_window(sentence, window_size, pad_token='<pad>'):
    window = [pad_token] * window_size
    return window + sentence + window
  #x = [pad_window(s, window_size=window_size) for s in x]

  def convert_token_to_indices(sentence, word_to_ix, default_token='<unk>'):
    return [word_to_ix.get(token, word_to_ix[default_token]) for token in sentence]

  x = [convert_token_to_indices(s, word_to_ix) for s in x]

  pad_token_ix = word_to_ix['<pad>']

  lengths = [len(s) for s in x]
  lengths = torch.LongTensor(lengths)
  #lengths = lengths.to('cuda')

  #print(x)
  x = [torch.LongTensor(x_i) for x_i in x]
  #x = torch.LongTensor(x)
  #print('x is:', x)
  x_padded = nn.utils.rnn.pad_sequence(x, batch_first=True, padding_value=pad_token_ix)
  x_padded = x_padded.to('cuda')
  #print('padded x is: ', x_padded)
  #print(x_padded)

  y = [convert_token_to_indices(s, label_to_ix, default_token='O') for s in y]
  pad_token_ix = label_to_ix['O']
  y = [torch.LongTensor(y_i) for y_i in y]
  y_padded = nn.utils.rnn.pad_sequence(y, batch_first=True, padding_value=pad_token_ix)
  y_padded = y_padded.to('cuda')
  #y = [label_to_ix.get(y_i) for y_i in y]
  #print(y)
  #y = [y_i for y_i in y]
  #y = new_y
  #y = torch.FloatTensor(y)
  #print(device)
  #y = y.to(device)
  #print('is y saved to cuda?:', y.is_cuda)
  #y_padded = nn.utils.rnn.pad_sequence(y, batch_first=True, padding_value=0)

  return x_padded, y_padded, lengths

In [ ]:
word_to_ix = {}
word_to_ix['<pad>'] = len(word_to_ix)
word_to_ix['<unk>'] = len(word_to_ix)
for sentence, tags in training_data:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

tag_to_ix = {'O': 0, 'B-DNA': 1, 'I-DNA': 2, 'B-RNA': 3, 'I-RNA': 4, 'B-protein': 5, 'I-protein': 6,
             'B-cell_line': 7, 'I-cell_line': 8, 'B-cell_type': 9, 'I-cell_type': 10, START_TAG: 11, STOP_TAG: 12}

In [ ]:
BATCH_SIZE = 13
shuffle = False
window_size = 2

#print(type(tag_to_ix))
collate_fn = partial(custom_collate_func, window_size=window_size, word_to_ix=word_to_ix, label_to_ix=tag_to_ix)

def examples_to_zip(data):
  pairs = []
  for example in data.examples:
    pairs.append((example.text, example.label))
  return pairs

#zipped_train_data = examples_to_zip(train_data)

data_loader = DataLoader(zipped_train_data, batch_size=BATCH_SIZE, shuffle=shuffle, collate_fn=collate_fn)

In [ ]:
INPUT_DIM = len(word_to_ix)
#print(INPUT_DIM)
EMBEDDING_DIM = 100
HIDDEN_DIM = 100
OUTPUT_DIM = 1
LEARNING_RATE = 0.9e-3

#model = LSTM(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)
#model = batchLR(INPUT_DIM, EMBEDDING_DIM, OUTPUT_DIM)
#print(type(tag_to_ix))
model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,268,082 trainable parameters


In [ ]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE)

In [ ]:
criterion = nn.BCEWithLogitsLoss()

In [ ]:
model = model.to(device)
#print(next(model.parameters()).is_cuda)
#model_batch = model_batch.to(device)
criterion = criterion.to(device)

In [ ]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
from tqdm import tqdm
def batch_train(model, loader, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch_input, batch_labels, batch_lengths in tqdm(loader, desc="Training...", total=len(loader)):
        
        optimizer.zero_grad()
        #sentence_in = prepare_sequence(sentence, word_to_ix)
        #targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long)
        #loss = model.neg_log_likelihood(sentence_in, targets)

        predictions = model(batch_input, batch_lengths)
        
        

        loss = model.neg_log_likelihood(batch_input, batch_labels)
        
        acc = binary_accuracy(predictions, batch_labels)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    
    return epoch_loss / len(loader), epoch_acc / len(loader)

In [ ]:
def batch_evaluate(model, loader, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch_inputs, batch_labels, batch_lengths in loader:

            predictions = model(batch_inputs, batch_lengths)
            
            loss = criterion(predictions, batch_labels)
            
            acc = binary_accuracy(predictions, batch_labels)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(loader), epoch_acc / len(loader)

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 11

best_valid_loss = float('inf')
model = model.to(device)
for epoch in range(N_EPOCHS):

    start_time = time.time()

    #train_loss, train_acc = train(model, test_iterator, optimizer, criterion)
    #valid_loss, valid_acc = evaluate(model, test_iterator, criterion)
    
    train_loss, train_acc = batch_train(model, data_loader, optimizer, criterion)
    valid_loss, valid_acc = batch_evaluate(model, data_loader, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Training...:   0%|          | 0/1427 [00:00<?, ?it/s]


RuntimeError: ignored